In [1]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\onkar\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
#  import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\onkar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [5]:
print('Number of positive tweets:', len(all_positive_tweets))
print('Number of negative tweets:', len(all_negative_tweets))

print('\nThe type of all_positive_tweets is:', type(all_positive_tweets))
print('The type of a tweet entry is:', type(all_negative_tweets[0]))

Number of positive tweets: 5000
Number of negative tweets: 5000

The type of all_positive_tweets is: <class 'list'>
The type of a tweet entry is: <class 'str'>


# Splitting Data
#### Train : Test :: 80 : 20

In [6]:
n = int(len(all_positive_tweets)*0.8)
test_pos = all_positive_tweets[n:]
train_pos = all_positive_tweets[:n]
test_neg = all_negative_tweets[n:]
train_neg = all_negative_tweets[:n]

In [7]:
train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [8]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_statements(statement):
    """Process statement function.
    Input:
        statement: a string containing a statement
    Output:
        statements_clean: a list of words containing the processed statement

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    # remove stock market tickers like $GE
    statement = re.sub(r'\$\w*', '', statement)
    
    # remove old style retweet text "RT"
    statement = re.sub(r'^RT[\s]+', '', statement)
    
    # remove hyperlinks
    statement = re.sub(r'https?:\/\/.*[\r\n]*', '', statement)
    
    # remove hashtags
    # only removing the hash # sign from the word
    statement = re.sub(r'#', '', statement)
    
    # tokenize statements
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    statement_tokens = tokenizer.tokenize(statement)

    statements_clean = []
    for word in statement_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # statements_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            statements_clean.append(stem_word)

    return statements_clean

In [9]:
def get_freqs(data):
    """Process statement function.
    Input:
        data: an array of strings containing a statement
    Output:
        freq_dict: a dictionary of words representing their frequencies

    """
    freq_dict = {}
    for i in data:
        for word in process_statements(i):
            if word in freq_dict:
                freq_dict[word]+=1
            else:
                freq_dict[word]=1
    return freq_dict

In [10]:
dict_of_positives = get_freqs(train_pos)
dict_of_negatives = get_freqs(train_neg)

In [11]:
pos = dict_of_positives.keys()
neg = dict_of_negatives.keys()

res = pos | neg
len(res)

9085

In [12]:
resultant_dict = {}
for i in res:
    pos_value = dict_of_positives.get(i,0)
    neg_value = dict_of_negatives.get(i,0)
    x = pos_value / (pos_value+neg_value)
    
    resultant_dict[i] = x

In [13]:
def check(s):
    cleaned = process_statements(s)
    
    arr = []
    for i in cleaned:
        x = resultant_dict.get(i, 0.5)
        arr.append(x)
        
    print(cleaned)
    print(np.mean(arr))
    
    return np.mean(arr)

In [14]:
s = 'good'
check(s)

['good']
0.6970802919708029


0.6970802919708029

In [15]:
import pickle

In [16]:
def save_data(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

In [17]:
def load_data(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [18]:
file = 'model_1.pkl'
# save_data(file, resultant_dict)

In [19]:
# x = load_data(file)

In [20]:
# x

In [21]:
pos_values = []
for i in test_pos:
    val = check(i)
    pos_values.append(val)

['bro', 'u', 'wan', 'cut', 'hair', 'anot', 'ur', 'hair', 'long', 'liao', 'bo', 'sinc', 'ord', 'liao', 'take', 'easi', 'lor', 'treat', 'save', 'leav', 'longer', ':)', 'bro', 'lol', 'sibei', 'xialan']
0.491346383781463
['back', 'thnx', 'god', "i'm", 'happi', ':)']
0.6153863338241786
['thought', 'ear', 'malfunct', 'thank', 'good', 'clear', 'one', 'apolog', ':-)']
0.6680242280845263
['stuck', 'centr', 'right', 'clown', 'right', 'joker', 'left', '...', ':)']
0.42509515743363446
['happi', 'friday', ':-)']
0.9364804469273743
['follow', ':)', 'x']
0.717712043431693
['teenchoic', 'choiceinternationalartist', 'superjunior', 'fight', 'oppa', ':D']
0.673611111111111
['birthday', 'today', 'birthday', 'wish', 'hope', "there'", 'good', 'news', 'ben', 'soon', ':-)']
0.5791146550619642
['good', 'morn', ':-)', 'friday', '\U000fec00', 'plan', 'day', 'current', 'play', 'shop', '...']
0.6519556093022684
['happi', 'friday', ':)']
0.9362464466933741
['3', 'good', 'nigth', ':)', 'estoy', 'escuchando', 'enemi'

c:\users\onkar\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\onkar\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['soon', '...', ':)']
0.6441499938552984
['sure', ':)']
0.7932997932997934
['look', 'fun', 'kik', 'thencerest', '547', 'kik', 'kiksext', 'sex', 'followback', 'l4l', 'indiemus', 'kikgirl', ':)']
0.5754341661190281
['goodnight', 'guy', ':-)', 'rememb', 'tomorrow', 'brand', 'new', 'day', 'fresh', 'start', 'anoth', 'chanc']
0.6391676041287943
['bom-dia', ':)', 'apod', 'ultraviolet', 'ring', 'm31', '2015', 'jul', '24']
0.560107185107185
["i'm", 'play', 'brain', 'dot', 'braindot']
0.6930006049606776
['thank', 'follow', 'great', 'day', ':)']
0.785660303651618
['oooh', 'love', 'hope', 'fantast', 'time', ':)']
0.703633657234063
['yawn', 'good', 'morn', 'everyon', 'wag', 'tail', 'everyon', 'today', ':D']
0.7074619268175775
['congrat', 'launch', ':D']
0.8774509803921569
['happi', 'wonder', 'birthday', 'love', '💟', 'beauti', 'day', 'world', ':)']
0.7976175322832277
['ftw', ':)']
0.49964899964899967
['ah', 'see', 'song', 'prefer', 'take', 'maman', ':)']
0.5307258390778075
['hi', 'hayley', ':-)', 'a

0.5497329595052772
['far', 'lfc', 'fan', 'make', 'expert', 'spot', 'mental', 'weak', 'lack', 'consist', ':-)']
0.5805994402816647
['tope', ':)']
0.7496489996489997
['good', 'night', 'sweet', 'dream', 'xxoo', '♡', '♥', '♡']
0.5847405559768698
['littl', 'finger', ':D']
0.6768115942028986
['love', ':)', '13', 'photo', 'explain', 'ukrain', 'russia']
0.7900448910301127
['friday', ':)']
0.9546489996489997
['hey', 'fam', 'vote', 'alreadi', ':)', 'gotta', 'win', 'boy', 'teenchoic']
0.5629403825055999
['hi', 'jane', 'silver-wash', 'fritillari', ':)']
0.6690903690903691
['pleasur', ':)']
0.9996489996489997
['great', 'day', 'hunni', ':)']
0.7549723292075408
['good', 'night', 'moon', ':)', 'pandora']
0.7642756582537604
['2', 'ap', 'studi', 'hall', 'haha', 'great', ':)']
0.5999628194202667
['follow', 'everyon', 'back', 'teamfollowback', ':-)']
0.7420242218609363
['ha', 'almost', 'gave', 'fuck', ':)', 'boy', 'trash', 'wast', 'time', "that'", 'sure']
0.4721719720047718
['amaz', 'pic', 'today']
0.6226

['thank', ':)']
0.9210536819232471
['jumma', 'mubarak', 'stay', 'bless', ':)']
0.8751983532471337
["we'r", 'look', 'forward', 'abl', 'spend', 'time', ':)', 'enjoy', 'iflix', ':)']
0.6672734940713086
['tgif', 'weekend', '...', 'enjoy', ':)']
0.7937373644599374
['im', 'thank', 'someon', 'like', 'life', 'inspir', 'better', 'everyday', 'love', 'mind', 'follow', ':)']
0.6144972287420853
['becom', 'better', 'atrack', 'better', ':)']
0.6585450458689895
['first', 'third', 'look', 'promis', 'els', 'fail', 'make', 'decis', 'go', ':)']
0.5764243846362264
['crazi', 'girlfriend', 'like', ':-)', '(-:', 'jesu', 'christ']
0.6193352137603008
['morn', ':)', 'gd', 'day', 'store', "i'm", 'badminton', '2', "littl'un", 'fabfriday']
0.6733361511950124
['kid', 'insid', 'want', 'one', ':-)']
0.5184424028143408
['imran', 'khan', 'hero', ':)', 'realli', 'ikprideofpak']
0.6983050906127829
['janjua', 'friend', ':)']
0.6902421902421901
['sorri', 'fuck', 'eric', 'think', 'he', 'funni', ':)']
0.516550082109503
['say'

['could', 'make', "foxy'", 'song', 'version', 'instrument', 'form', 'sound', 'epic', 'beat', 'alon', ':)']
0.5624728130442702
['second', 'well', 'done', ':)']
0.7475347356156952
['think', 'hard', 'bed', ':p']
0.525946275946276
['tomorrow', 'goldcoast', ':)']
0.681052549473602
['like', 'way', 'said', 'gal', 'lelomustfal', 'beautiful', 'way', ':)']
0.6741420005139518
['happi', 'save', 'day', ':)', 'hope', 'enjoy', 'meal', 'sam']
0.6101518792307501
['thank', 'min', ':)']
0.8084802323932759
['5g', 'liker', ':)', 'like', 'fast']
0.5512146134097353
['like', 'morn', 'got', 'dress', 'bought', ':)', "i'm", 'total', 'love', '♡', 'newdress', 'happi', '…']
0.5837579118254479
['ff', 'could', 'anyon', 'resist', 'beauti', 'smile', ':-)', 'quacketyquack']
0.7161048929141035
['u', 'mean', 'hide', 'fot', ':p']
0.5402907605430733
['aww', 'thankyou', ':)']
0.7213777213777214
['ok', 'good', 'night', 'wish', 'troy', 'ugli', 'met', 'today', ':)', '):', 'ok', 'today', 'fun', "i'm", 'excit', 'tmrw']
0.58247775

['ye', 'alway', 'selfish', 'fuck', ':)']
0.48077321360903447
['hey', 'like', 'fnaf', 'check', 'youtub', 'channel']
0.6184131285757302
['want', 'creat', 'first', 'bboy', 'cardgam', 'pixelart', 'gamedesign', ':D', 'think', 'indiedev', 'pixel_daili', 'gamedev']
0.5917515518893471
['sound', 'still', 'like', 'good', 'idea', ':)']
0.6271759576606223
['limit', 'plateau', 'must', 'stay', 'must', 'go', 'beyond', ':)', 'bruce', 'lee']
0.6212697050615372
['man', 'sing', 'rain', ':)']
0.5045864045864046
['laguna', ':)', 'see']
0.5330993330993331
[':)', 'time', 'nhi', 'tha', 'offic', 'work', 'bahot', 'tha', '10', 'baje', 'raat', 'ko', 'liya', 'tha', 'hath', 'tri', 'creat', 'best', '2', 'ghant', 'itna', 'hi', 'bana', 'paya']
0.5264610443529208
["that'", 'true', 'quit', 'whenev', 'want', ':D']
0.5497590213504014
['thank', ':)', 'weekend', 'tri', 'best', ':)']
0.784786528901057
['u', 'like', 'uta', 'read', 'manga', ':-)', 'love', 'even', ':-)', 'haha', 'yeah']
0.5717789414420296
['famili', 'hang', ':)

["i'm", 'insecur', 'tonight', ':-)']
0.5266830870279147
['<3', 'guy', ':)']
0.7922469667230675
['dan', 'god', 'bless', 'meet', 'greet', 'soon', ':)']
0.661428000713715
['mom', ':)', 'horror', 'movi']
0.602765676295088
['oh', 'hell', 'yeah', ':)', "i'll", 'expect', 'text', 'next', 'tuesday']
0.5255812034598466
['welcom', 'sabah', ':-)']
0.9717514124293786
['match', 'day', 'bitchesss', 'real', 'madrid', 'vs', 'man', 'shitti', ':D']
0.4545670809236701
['squash', 'commit', 'still', 'make', '200', 'commit', ':)']
0.4565245158523819
['west', 'countri', 'weekend', 'next', 'ride', 'day', 'tuesday', ':)', 'sunni', 'weekend', 'around']
0.6838259367356898
['ourdaughtersourprid', 'mani', 'mani', 'congratul', 'papa', 'ji', ':)']
0.8420575563432706
['thank', 'becca', ':)']
0.780702454615498
['love', 'new', 'song', 'delta', 'rock', ':)']
0.6599098422584545
['saw', 'thought', 'might', 'need', 'upgrad', 'girl', 'get', 'bigger', ':)']
0.5758184132658185
['first', 'love', 'wanna', 'fuck', 'late', 'night'

['love', 'airport', ':-)', ':-)']
0.7365900383141762
['love', ':)', 'want', 'corn', 'chip', ':)']
0.749064578822928
['thank', ':)', 'tricki', 'bt', 'v', 'import', 'subject']
0.5774439091209277
['hah', '...', ':D', 'say', 'sorri', '...']
0.5782871279924325
['be-shak', ':p']
0.75
[':)']
0.9992979992979993
['happi', 'birthday', 'miss', 'chenoweth', 'hope', 'great', 'one', '...', 'oodl', 'cake', ':)']
0.5939939754770627
['happi', 'birthday', 'gorgeou', 'hope', "i'll", 'see', 'fair', ':)']
0.7154793693390117
['im', 'go', 'bed', 'love', 'hailey', 'milk', 'craźi', 'sęxxxÿ', 'cøôl', 'runway', 'look', ':)']
0.519995500925089
['feel', 'someon', 'share', 'review', 'work', 'hard', ':D']
0.5781772583861328
['gooodnight', 'love', 'boy', 'iv', 'ri', 'zoe', 'kati', 'jayci', 'jen', ':)']
0.6121986654970163
['well', 'bud', 'guess', "i'm", 'go', 'bed', 'goodnight', 'guy', 'see', 'morn', 'whatev', 'woke', ':)']
0.5877197500991492
['thank', 'follow', 'great', 'day', ':)']
0.785660303651618
['thank', ':)']


['wow', 'luxord', 'look', 'realli', 'amaz', 'new', 'kingdom', 'heart', '3', 'promo', 'art', ':)']
0.5945339974434797
['new', 'potato', 'garden', 'hundr', 'dig', ':)']
0.48906048906048905
['love', ':D']
0.8620689655172413
['thank', 'love', 'weekend', 'everyon', ':-)']
0.8050270897786165
['well', 'total', 'blew', 'mind', 'morn']
0.6886029060453612
['cours', "i'v", 'cite', 'academ', 'paper', 'feel', "i'v", 'arriv', ':-)']
0.44539495618037983
['life', 'smile', ':)']
0.7974766224766224
['thank', 'info', ':)']
0.9170660909791345
['sure', 'watch', 'pokiri', '1nenokkadin', 'tym', 'fav', ':-)', 'bajrangibhaijaanhighestweek', '1']
0.6895939799983797
['yang', 'retweet', ':)', ':D']
0.9581578331578332
["who'", 'still', 'awak', ':)']
0.575042721319317
['heritag', 'sword', 'made', 'wood', ':D']
0.6288135593220339
['right', 'thx', 'andrew', 'great', 'day', ':)']
0.8206631496891558
['true', ':)']
0.7496489996489997
['beleaf', 'u', ':-)']
0.6579558652729385
['cool', 'video', 'love', '...', ':)', 'thank

In [22]:
neg_values = []
for i in test_neg:
    val = check(i)
    neg_values.append(val)

['help', '...', 'stop', 'cri', ':(']
0.2688610771158464
['otl', 'nevermind', ':(', 'least', 'got', 'jeon']
0.31775775772058784
['soon', 'tweet', 'plant', 'claw', 'thigh', 'traction', 'zoom', 'away', ':(']
0.4497169347715199
['damnit', ':(']
0.25013646288209607
['use', 'pri', 'pv', '...', 'wish', 'could', 'reliv', 'day', 'becom', 'nyc', 'pv', 'buy', 'way', 'commun', 'nyc', 'usa', 'klm', ':(']
0.5874243583897542
['realli', 'hot', ':-(']
0.36006257226416377
['monday', ':(']
0.25013646288209607
['go', 'stop', 'breakfast', 'earli', 'might', 'want', 'remov', '11am', 'websit', 'even', "mcd'", 'pull', 'trick', ':(']
0.46944233888795195
['mean', 'way', ':(', '3rd', 'load', 'hung']
0.29923008336368523
[':(', 'wtf', 'suppos', 'without']
0.26611597679913823
['headach', 'strike', ':(']
0.1667576419213974
['english', 'weather', 'need', 'fix', ':(']
0.28473122607947937
['live', 'fam', 'bam', 'cough', ':(']
0.5478323629306162
['absolut', 'gut', 'jame', 'bay', 'ticket', 'sold', 'manchest', ':(']
0.2991

['neighbor', 'let', 'sleep', ':-(']
0.36947898799313894
['back', 'delhi', ':(']
0.4768351613012423
['feet', 'cold', 'point', 'take', 'sock', 'okay', 'stop', ':(']
0.30898045545044583
["can't", 'belieb', 'rain', ':(']
0.31419765415985623
['sinc', 'model', 'stop', 'stock', 'boot', "can't", 'get', "can't", 'even', 'order', 'onlin', 'tell', 'unsaf', ':(']
0.3508352629808025
['summer', 'holiday', 'great', "i'm", 'bore', 'alreadi', ':(']
0.3859729033446348
['oh', 'plan', 'stream', 'halo', 'xbox', 'one', 'today', ':(']
0.40890594602348423
['drop', 'dead', 'fred', 'use', 'favorit', 'movi', 'wish', ':-(']
0.4407147502256198
['rememb', '2', 'gaon', 'bad', '6', 'infnt', 'elig', 'apink', ':(', 'acub', "why'd", 'u']
0.4428088798506556
['sad', 'everi', 'time', 'apink', 'event', '..', 'understand', 'korean', ':(', '😭', '😭']
0.3329054384313825
[':(', 'fun']
0.3462903090359422
['bullshit', '...', ':(']
0.31541514028420026
[':(', 'know', 'happen', 'internet', 'light', 'connect', 'drop', 'thank', 'kei']


['pleas', 'god', 'wanna', 'go', 'work', ':(']
0.31010361437735856
['kinda', 'miss', 'pamela', 'good', 'anna', ':(']
0.3058646731939278
['pleas', 'woe', "can't", 'run', 'six', 'though', 'live', 'mexico', ':(']
0.36301856087929874
['idk', "i'm", 'get', 'mini', 'bu', 'idk', 'time', "i'm", 'go', 'get', ':(']
0.3076152978771913
['hi', 'harri', 'sorri', ':(', "we'r", 'work', 'mast', 'area', 'caus', 'intermitt', 'servic', 'work', 'complet']
0.5059313569812924
['look', 'fun', 'snapchat', 'janniecam', 'snapchat', 'kikm', 'webcam', 'snapchatm', 'kikhorni', 'musicbiz', 'hotmusicdeloco', ':(']
0.4422979323864636
['braxton', 'gone', 'pro', ':(', 'feel', 'bad', 'urban', 'unpreced', 'brainwash', 'see', 'tebow', 'alex', 'smith']
0.40056729160773
['wat', 'okaaay', 'huhu', ':(', 'sayanggg', 'find', 'yah', 'phone']
0.3269235253305219
['1', 'taxi', 'servic', '2', 'second', 'schedul', 'blog', 'post', '3', 'housework', 'bust', ':(']
0.5531028107378535
['mum', 'say', 'done', 'leav', 'step', 'dad', ':-(']
0.3

0.3650585210574957
['eric', 'made', 'still', 'tell', 'love', ':-(', 'fake', 'ass']
0.40819221376037723
['miss', 'itb', 'omigod', ':(']
0.26666979394104806
['need', 'job', 'learn', 'drive', 'fricken', 'health', 'put', 'back', ':(']
0.43613133095335793
['hi', 'unfortun', "we'r", 'unabl', 'locat', 'origin', 'complaint', ':(', 'pl', 'dm', 'us', 'product', 'detail', "we'll", 'get', 'back']
0.42007412668334365
['miss', 'boyfriend', ':-(']
0.022135416666666668
[':-(']
0.0
['ehem', '😏', '😏', 'haha', 'ala', 'yeke', ':(', 'okay', 'fun', 'kk', 'jumpa', 'next', 'time', '✈', '️', '🇺', '🇸', 'mayb', '😋']
0.4365974363978248
['come', 'home', 'empti', 'hous', 'ape', ':(']
0.4231391695445767
["i'v", 'shit', 'bestfriend', ':(']
0.3678315605807817
["i'm", 'gonna', 'inact', 'next', 'month', ':(']
0.25586586175402753
['miss', ':(']
0.03333958788209607
[':(', 'well', 'tri', 'someth', 'keep', 'mind', 'bad', 'thing', "we'r", 'alway']
0.46803806495464534
["i'v", 'never', 'seen', 'felt', 'snow', ':(']
0.236685841

['see', 'delph', 'anoth', 'kit', 'upset', ':(']
0.22933841692029466
['miss', ':(', 'haha', "x'", '😂', '😂', 'still', 'rememb', 'u', 'hate', 'xxx']
0.36961192984949415
['zayniscomingbackonjuli', '26', 'funni', ':(']
0.19743665249367962
["we'r", 'allow', 'cheat', 'meal', ':(']
0.19116569626394955
['fun', 'osaka', 'super', 'junior', ':(', 'would', 'love', 'watch', 'comeback', 'stage', 'though']
0.48621347864967834
['aytona', 'hala', 'guy', 'get', 'readi', 'na', 'mathird', 'wheel', ':(', 'jkjk']
0.39322109269869066
['worst', 'day', 'ever', ':(', 'pain', '...']
0.3416825486519624
['backtrack', '3', 'hour', ':(', '...', 'time', 'start', 'actual', 'util', 'list']
0.45921509169788094
['surviv', 'anoth', 'week', 'without', 'phone', ':(']
0.3635767726596646
['wish', ':(']
0.15774515853426999
['pat', 'jay']
0.75
['sad', 'stori', 'broh', ':(']
0.256020612393429
['uh', 'asleep', ':(']
0.09731319747695294
['need', 'boyfriend', 'rn', 'answer', 'calll', ':(']
0.25908099372169496
['icaru', '500', 'awn',

['that', 'push', 'emoji', 'give', 'soni', 'five', ':-(', ':-(', ':-(', ':-(', ':-(', ':-(']
0.20569539791507754
['late', ':(', 'followmeaaron']
0.2778687530325085
['tzelumxoxo', 'dont', 'realli', 'use', 'line', 'ipod', ':-(', 'use', 'whatsapp', 'actual']
0.373188341572957
['buy', 'someth', 'drank', 'caus', 'noth', ':(']
0.48730178507388017
['slept', 'day', "can't", 'sleep', ':(']
0.3106959699189128
[':-(', 'pleas', 'notic', 'mew']
0.27025283797729616
['peopl', ':(', 'r', 'u']
0.3600124031230246
['indic', 'sm', 'receiv', ':(']
0.30661585048866713
['got', 'regular', ':(']
0.2960093426016695
['kik', 'ouliv', '70748', 'kik', 'kikmenow', 'photo', 'babe', 'loveofmylif', 'brasileirao', 'viernesderolenahot', ':(']
0.2848756232281945
['feel', 'u', ':(']
0.2251200112593765
['longmorn', '30', 'suppos', 'replac', 'tobermori', '32', '...', 'begin', 'strong', 'tail', 'miss', ':(']
0.456609861459939
['wait', 'love', 'recuerda', 'tanto', 'bath', ':(']
0.5548010768657132
['sun', 'gone', ':(']
0.2096147

[':(', 'unfollow', 'like', '300', 'peopl', 'caus', 'follow', '1k', 'peopl', 'difficult']
0.38137976617050684
['enough', 'time', 'listen', 'artist', 'music', ':-(']
0.38398692810457513
['take', 'ice', 'creamist', 'pleasee', ':(']
0.29055920491118953
['alway', 'go', 'terribl', 'time', ':(']
0.3020110266441637
[':-(', 'challeng', 'though', 'pleas', 'check', 'fb', 'page', 'entri', 'rather', 'substitut', 'thank']
0.5068312243436757
["can't", 'even', 'see', 'hyungwon', ':(']
0.22135812332788501
['tell', 'pl', 'eat', ':-(']
0.217989417989418
['that', 'life', 'get', 'call', 'peopl', 'havent', 'seen', '20', 'year', 'alway', 'favour']
0.5034481774280877
['omg', ':(', 'everi', 'time', 'would', 'complain', 'skin', 'mom', 'everyon', 'get', 'pimpl', 'like', 'although']
0.48130820371799027
['letsfootbal', 'atk', 'greymind', '43', 'break', 'news', 'chri', 'gayl', 'say', 'cricket', '2-3', 'month', 'due', 'back', 'surgeri', ':(', 'cplt', '20', 'cp', '…']
0.29199639367746205
['miss', 'hannah', 'montana',

['apolog', 'annoy', 'much', 'snapchat', ':(']
0.3158997536295859
['adult', 'take', 'thing', 'seriou', 'realli', 'miss', 'kid', ':(']
0.27080444867856496
['facad', 'democraci', 'drop', 'quickli', ':(']
0.4700545851528385
['confus', 'brat', ':(']
0.24083171599547146
['someon', 'come', "i'll", 'make', 'popcorn', 'watch', 'movi', 'pl', 'cuddl', ':(']
0.31995049201141934
['final', 'home', 'drive', 'sinc', '5am', ':(']
0.2692855068673846
['tb', 'mr', "o'", 'class', ':(']
0.40561014070839396
['bade', 'fursat', 'se', 'usey', 'banaya', 'hai', 'usey', 'uppar', 'waal', 'ney', 'afso', 'hums', 'dur', 'hai', 'wo', ':(']
0.5062670578602619
['oh', 'realli', ':(', 'saw', 'gif', 'post', 'seen', 'realli', 'happi', 'mayb', 'peopl', "who'd", 'happi', 'naruhina']
0.44851317130198204
["can't", ':(']
0.0867286416530458
['that', 'english', 'namee', ':(', 'haiqal', 'isnt', 'english', 'name', 'rememb']
0.36422414296293865
['want', 'dont', 'paypal', ':(']
0.2479816172678197
["that'", '360hr', 'summer', 'end', '..

In [23]:
error_in_pos = 0
for i in pos_values:
    if i>0.5:
        error_in_pos+=1

error_in_pos = 100 - error_in_pos/len(pos_values)*100
error_in_pos

9.0

In [24]:
error_in_neg = 0
for i in neg_values:
    if i<=0.5:
        error_in_neg+=1

error_in_neg = 100 - error_in_neg/len(neg_values)*100
error_in_neg

6.799999999999997

In [25]:
check('')

[]
nan


nan

In [26]:
all_positive_tweets

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.',
 'Jgh , but we have to go to Bayan :D bye',
 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing 

In [27]:
check('this movie has no value addition to the person')

['movi', 'valu', 'addit', 'person']
0.5


0.5